In [ ]:
#This code is based on a tutorial provided by https://colab.research.google.com/drive/1hR0XHwW3rEpHHixyZqzQE-DyfDiZORnM#scrollTo=gFsCTp_mporB 

  #Allrigh.. First of all, we need to split up our data. We'll be training two models: one on the '0' - labeled data and another one on the '1' - labeled one

In [1]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia -y


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit=11.1
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.1.74        |       h6bb024c_0        1.19 GB  nvidia
    ------------------------------------------------------------
                                           Total:        1.19 GB

The following packages will be UPDATED:

  cudatoolkit        pkgs/main::cudatoolkit-11.0.221-h6bb0~ --> nvidia::cudatoolkit-11.1.74-h6bb024c_0



cudatoolkit-11.1.74  | 1.19 GB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing trans

In [38]:
!pip install transformers

TEST: Text generation using German GPT - 2:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2")

In [ ]:
from transformers import pipeline

pipe = pipeline('text-generation', model="dbmdz/german-gpt2",
                 tokenizer="dbmdz/german-gpt2")

text = pipe("Von toxischen Kommentaren halte ich", max_length=100)[0]["generated_text"]

print(text)

//TEST: Text Generation using German GPT - 2

First of all: Let's create dataframes with toxic and non-toxic data

In [1]:
import pandas as pd

In [2]:
df_toxic = pd.read_csv('data_fragments_base/toxic_base.csv', encoding= 'UTF-8')
df_toxic.head()

,comment_id,comment_text,Sub1_Toxic
0,3,"Schublade auf, Schublade zu. Zu mehr Denkleist...",1
1,6,@USER Schon mal was von Physik gehört?,1
2,8,@USER geh wieder pennen...dein Gesülze interes...,1
3,9,Wenn ich einen Konjunktiv verwende hat das ein...,1
4,10,Nach mir die Sintflut ☺,1


In [3]:
df_ntoxic = pd.read_csv('data_fragments_base/non_toxic_base.csv', encoding= 'UTF-8')
df_ntoxic.head()

,comment_id,comment_text,Sub1_Toxic
0,1,Ziemlich traurig diese Kommentare zu lesen. Ih...,0
1,2,"Sag ich doch, wir befeuern den Klimawandel. Ra...",0
2,4,Dummerweise haben wir in der EU und in der USA...,0
3,5,"""So lange Gewinnmaximierung Vorrang hat, wird ...",0
4,7,"Sollte es dann doch einen Klimawandel geben, d...",0


In [4]:
comments_toxic = df_toxic.copy()

In [ ]:
df.iloc[0]['comment_text']

We'll use the GPT - 2 Tokenizer

In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                         bos_token = '<|startoftext|>',
                                         eos_token = '<|endoftext|>',
                                         pad_token = '<|pad|>')

tokenizer.encode("Mushishi")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[44, 1530, 21644]

In [6]:
max_comment = max([len(tokenizer.encode(comment)) for comment in comments_toxic.comment_text.values])
print(f'The longest comment in the datase is {max_comment} tokens long')
max_comment = 1024

Token indices sequence length is longer than the specified maximum sequence length for this model (1901 > 1024). Running this sequence through the model will result in indexing errors


The longest comment in the datase is 2035 tokens long


In [7]:
!nvidia-smi

Sun Jun 27 12:30:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:09:00.0 Off |                  N/A |
| 27%   35C    P8    16W / 225W |  10441MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:0A:00.0 Off |                  N/A |
| 27%   38C    P8     7W / 225W |    313MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

Set batch size:

In [13]:
bs = 16

In [14]:
import torch
torch.manual_seed(42)
from torch.utils.data import Dataset

class ToxicDataset(Dataset):
    
    def __init__(self, txt_list, tokenizer, gpt2_type = "gpt2", max_length = max_comment):
        
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        
        for txt in txt_list:
            
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>',
                                      truncation = True,
                                      max_length = max_length,
                                      padding ="max_length")
            
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [15]:
from torch.utils.data import random_split

dataset = ToxicDataset(comments_toxic.comment_text.values, tokenizer, max_length=max_comment)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {train_size} samples for training, and {val_size} samples for validation testing'

'There are 897 samples for training, and 225 samples for validation testing'

In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), #training sampling random
            batch_size = bs
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), #order for testing doesn't matter
            batch_size = bs
        )

Now, let's fine-tu~ne!

In [22]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.cuda()


seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [18]:
torch.cuda.empty_cache()

In [21]:
!nvidia-smi

Mon Jun 28 09:03:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:09:00.0 Off |                  N/A |
| 27%   34C    P8    16W / 225W |  11014MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:0A:00.0 Off |                  N/A |
| 27%   37C    P8     9W / 225W |    313MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G